In [1]:
import os
import pandas as pd
import json

In [2]:
%run utils/__init__.py

In [3]:
%run metrics/__init__.py

In [4]:
classification = False

In [5]:
results_folder = os.path.join(WORKSPACE_DIR,
                              'classification' if classification else 'report_generation',
                              'results')
run_names = os.listdir(results_folder)
run_names.remove('debug')
run_names

['0713_140436_h-lstm-att_lr0.0001_resnet-50',
 '0714_122624_lstm_lr0.0001_densenet-121',
 '0714_100643_lstm_lr1e-05_densenet-121',
 '0712_204300_lstm-att_lr0.001_densenet-121',
 '0713_145700_h-lstm-att_lr0.001_resnet-50',
 '0712_183409_lstm-att_lr0.0001_densenet-121',
 '0712_162849_lstm-att_lr1e-05_densenet-121',
 '0713_130954_h-lstm-att_lr1e-05_resnet-50']

In [6]:
all_results = []

for run_name in run_names:
    filepath = os.path.join(results_folder, run_name, 'metrics.json')
    
    if not os.path.isfile(filepath):
        print('Not found for: ', run_name)
        continue

    with open(filepath, 'r') as f:
        results = json.load(f)

    all_results.append((run_name, results))

len(all_results)

8

In [7]:
def create_results_df(key):
    run_names = [run_name for run_name, _ in all_results]
    
    results_dicts = [d[key] for _, d in all_results]
    
    df = pd.DataFrame(results_dicts, index=run_names)
    return df

In [14]:
results_df_val = create_results_df('val')
results_df_test = create_results_df('test')

### Report generation

In [17]:
results_df_test = results_df_test.loc[results_df_test.index.str.contains('_lstm_')]

In [9]:
results_df_val

,loss,word_acc,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0713_140436_h-lstm-att_lr0.0001_resnet-50,3.392219,0.105569,0.226439,0.141758,0.095739,0.063009,0.131736,0.244193,0.164773
0714_122624_lstm_lr0.0001_densenet-121,6.122686,0.092413,0.483759,0.311496,0.208980,0.156559,0.290198,0.362386,0.585445
0714_100643_lstm_lr1e-05_densenet-121,4.413955,0.107711,0.213193,0.096728,0.021586,0.000001,0.082877,0.209026,0.070247
0712_204300_lstm-att_lr0.001_densenet-121,12.876278,0.081662,0.451238,0.288113,0.205312,0.157811,0.275619,0.338023,0.822703
0713_145700_h-lstm-att_lr0.001_resnet-50,5.020334,0.112234,0.409183,0.242419,0.153279,0.096048,0.225232,0.310534,0.410248
0712_183409_lstm-att_lr0.0001_densenet-121,8.984263,0.065020,0.425976,0.281001,0.192660,0.141549,0.260297,0.345751,0.606811
0712_162849_lstm-att_lr1e-05_densenet-121,4.531137,0.084763,0.269817,0.165678,0.105352,0.072291,0.153284,0.256767,0.053960
0713_130954_h-lstm-att_lr1e-05_resnet-50,2.525155,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
results_df_test

,loss,word_acc,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0714_122624_lstm_lr0.0001_densenet-121,6.342473,0.090482,0.496526,0.322851,0.212395,0.153838,0.296403,0.363302,0.477281
0714_100643_lstm_lr1e-05_densenet-121,4.322180,0.105179,0.217396,0.098144,0.015933,0.004586,0.084015,0.208349,0.070824


In [19]:
print_columns = [c for c in results_df_test.columns if c != 'loss' and 'cm' not in c]
print((results_df_test).to_latex(columns=print_columns,
                               float_format='%.2f',
                              ))

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  word\_acc &  bleu1 &  bleu2 &  bleu3 &  bleu4 &  bleu &  rougeL &  ciderD \\
\midrule
0714\_122624\_lstm\_lr0.0001\_densenet-121 &      0.09 &   0.50 &   0.32 &   0.21 &   0.15 &  0.30 &    0.36 &    0.48 \\
0714\_100643\_lstm\_lr1e-05\_densenet-121  &      0.11 &   0.22 &   0.10 &   0.02 &   0.00 &  0.08 &    0.21 &    0.07 \\
\bottomrule
\end{tabular}



### Classification

In [38]:
results_df_val = results_df_val.loc[results_df_val.index.str.contains('covid-kaggle')]
results_df_test = results_df_test.loc[results_df_test.index.str.contains('covid-kaggle')]

In [30]:
results_df_val.sort_values(['acc', 'recall_covid', 'prec_covid'], ascending=False)

,loss,acc,prec_covid,prec_pneumonia,prec_normal,recall_covid,recall_pneumonia,recall_normal,spec_covid,spec_pneumonia,spec_normal,cm
0711_124133_covid-x_vgg-19_lr1e-06_os,1.645402,0.924352,0.895833,0.915572,0.931846,0.914894,0.895413,0.944724,0.996271,0.946619,0.907095,"[[42, 3, 2], [2, 491, 52], [2, 48, 746]]"
0709_121201_covid-x_densenet-121_lr1e-06_os,1.090467,0.904179,0.952381,0.899048,0.904994,0.851064,0.866055,0.933417,0.998509,0.937129,0.868243,"[[39, 2, 6], [1, 476, 68], [1, 56, 739]]"
0710_191443_covid-x_vgg-19_lr1e-06,1.329106,0.770893,0.975610,0.638118,0.967181,0.851064,0.970642,0.629397,0.999254,0.644128,0.971284,"[[39, 5, 3], [1, 529, 15], [0, 313, 483]]"
0708_192944_covid-x_densenet-121_lr1e-06,13.103516,0.392651,0.000000,0.392651,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,"[[0, 47, 0], [0, 545, 0], [0, 796, 0]]"


In [39]:
results_df_test.sort_values(['acc', 'recall_covid', 'prec_covid'], ascending=False)

,loss,acc,prec_covid,prec_pneumonia,prec_normal,recall_covid,recall_pneumonia,recall_normal,spec_covid,spec_pneumonia,spec_normal,cm
0704_190958_covid-kaggle_resnet-50_lr1e-06_os,0.002729,0.975779,0.875000,0.992366,0.977612,1.000000,0.970149,0.977612,0.988806,0.993548,0.980645,"[[21, 0, 0], [1, 131, 2], [2, 2, 130]]"
0704_015027_covid-kaggle_densenet-121_lr1e-06_os,0.008223,0.965398,1.000000,0.984375,0.942857,1.000000,0.940299,0.985075,1.000000,0.987097,0.948387,"[[21, 0, 0], [0, 127, 7], [0, 2, 132]]"
0704_114753_covid-kaggle_densenet-121_lr1e-06_noig_os,0.067778,0.951557,0.724138,0.984127,0.970149,1.000000,0.925373,0.970149,0.970149,0.987097,0.974194,"[[19, 2, 0], [5, 125, 4], [2, 2, 130]]"
0704_232547_covid-kaggle_resnet-50_lr1e-06,1.058202,0.903114,0.909091,0.975000,0.848101,0.476190,0.873134,1.000000,0.996269,0.980645,0.845161,"[[7, 5, 9], [1, 120, 13], [0, 0, 134]]"
0704_062601_covid-kaggle_densenet-121_lr1e-06,2.662862,0.757785,0.800000,0.870968,0.701571,0.190476,0.604478,1.000000,0.996269,0.922581,0.632258,"[[4, 13, 4], [0, 92, 42], [0, 0, 134]]"
0704_161918_covid-kaggle_densenet-121_lr1e-06_noig,3.421237,0.737024,0.333333,0.879121,0.676923,0.047619,0.597015,0.985075,0.992537,0.929032,0.593548,"[[0, 12, 9], [0, 87, 47], [0, 2, 132]]"
0705_055411_covid-kaggle_resnet-50_lr1e-06_noig,5.772729,0.733564,0.000000,0.751938,0.718750,0.000000,0.723881,0.858209,1.000000,0.793548,0.709677,"[[0, 16, 5], [0, 100, 34], [0, 28, 106]]"
0705_013805_covid-kaggle_resnet-50_lr1e-06_noig_os,0.162184,0.692042,0.287879,0.788462,0.831933,0.904762,0.611940,0.738806,0.824627,0.858065,0.870968,"[[18, 2, 1], [30, 88, 16], [11, 25, 98]]"


In [40]:
print_columns = [c for c in results_df_test.columns if c != 'loss' and 'cm' not in c]
print((results_df_test*100).to_latex(columns=print_columns,
                               float_format='%.2f',
                              ))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &   acc &  prec\_covid &  prec\_pneumonia &  prec\_normal &  recall\_covid &  recall\_pneumonia &  recall\_normal &  spec\_covid &  spec\_pneumonia &  spec\_normal \\
\midrule
0704\_232547\_covid-kaggle\_resnet-50\_lr1e-06         & 90.31 &       90.91 &           97.50 &        84.81 &         47.62 &             87.31 &         100.00 &       99.63 &           98.06 &        84.52 \\
0704\_062601\_covid-kaggle\_densenet-121\_lr1e-06      & 75.78 &       80.00 &           87.10 &        70.16 &         19.05 &             60.45 &         100.00 &       99.63 &           92.26 &        63.23 \\
0704\_015027\_covid-kaggle\_densenet-121\_lr1e-06\_os   & 96.54 &      100.00 &           98.44 &        94.29 &        100.00 &             94.03 &          98.51 &      100.00 &           98.71 &        94.84 \\
0705\_013805\_covid-kaggle\_resnet-50\_lr1e-06\_noig\_os & 69.20 &       28.79 &           78.85 &        83.19 &         90.48 &             6